# Data Engineering in Energy
### Albieri M. Alana, Estudiante de Ing Informática URBE
### 2026
***
## Week 2 Workbook
***

##**1.	Define and describe the following:**

###**a.	Feature Imputation**

Al hablar de Feature Imputation hacemos referencia a que hacer cuando tenemos valores nulos o faltantes en nuestro dataset, tenemos por ejemplo la opcion de tomar valores de tendencia central como la media para rellenarles, sin embargo, debemos tomar en cuenta que al tener outliers es posible que esos datos rellenados tengan sesgos. Otra opcion es predecirles usando regresiones lineales, bajo costo computacional sin embargo es muy sensible ya que es valga la redundancia lineal, si se quiere mayor exactitud esta el algoritmo knn que toma el promedio de los vecinos mas cercanos. Cada algoritmo tiene sus pros y contras, su implementacion dependera de las necesidades del negocio.

###**b.	Feature Transformation**

El Feature Transformation esta enfocado a la transformacion de los datos segun los requerimientos y escalas del mismo, como ejemplo tengamos una correlacion entre edad y salarios, dos tipos de enfermedades en grados o similar, basicamente comparando dos variables con cantidades medibles muy diferentes llevarles a un mismo nivel. Una de las estrategias mas famosas es la de normalizacion de min-max que consiste en llevarle de la cantidad inicial a su respectivo entre 0-1.

###**c.	Feature Ranking**

Basicamente se centra en asignar prioridades en nuestras carecteristicas para nuestro modelo, determinando cual tiene mayor importancia en comparacion a el resto, creando un "Ranking" de importancias, de la mayora a menor.

###**d.	Epistemic vs. Aleatoric Uncertainty**

Consiste en dos formas distintas de clasificar la incertidumbre, la incertidumbre epistemica tiene como base el desconocimiento. Es decir, se basa en que desconoce un hecho para calcular la incertidumbre sin emabrgo estos niveles de incertidumbre pueden bajar a medida que conozcamos mas del hecho a diferencia de la incertidumbre aleatoria que como bien dice el nombre es completamente aleatoria y fija, la misma no depende de nosotros para bajar o aumentar.

###**e.	Curse of Dimensionality**

Podemos pensar que mientras mas variables tenemos en nuestro modelo, es mejor, ¿o no?. No, por el contrario debemos calcular la cantidad viable de variables para que la misma no se vea afectada al tener pocos datos y una cantidad excesiva de varibles. Es ese concepto reside la "Curse of Dimensionality", muchas variables y pocos datos.

###**f.	Causation vs. Correlation**

Una fuerte correlacion conlleva siempre una causalidad fija cierto? No, al hablar de correlacion tomamos en cuenta que dos variables se mueven a un ritmo similar, sin embargo no siempre el hecho de A implica B. Supongamos que tenemos una fuerte correlacion entre paraguas y accidentes de vehiculos, casi de 1, eso quiere decir que los paraguas son causantes de los accidentes? no, tenemos una variable oculta en este caso la lluvia que seria la variable C quien conecta e influye en las otras 2. Para validar que dos variables con fuerte correlacion tengan causas similares se suele hacer testing A/B.

***
##**2.	Feature Imputation**

###**a.	Load the dataset as a pandas dataframe and provide a summary of the features using “pd.describe()”. How many NaN’s are there and what percent of values in the dataframe are NaNs? Pay special attention to the minimum/maximum and mean/stdv values of each feature.**

In [1]:
# Primero vamos a importar las librerias requeridas y las bases de datos

import pandas as pd

df_original = pd.read_csv("https://github.com/Albieri151/Ejercicios-Data-Science-Petroleo/raw/refs/heads/main/Week%202/unconv_MV_v2_missing.csv")
df_original.head()

,Por,LogPerm,AI,Brittle,TOC,VR,Production
0,15.91,1.67,3.06,14.05,1.36,1.85,177.381958
1,15.34,1.65,2.60,31.88,1.37,1.79,1479.767778
2,20.45,2.02,3.13,63.67,1.79,2.53,4421.221583
3,11.95,1.14,3.90,58.81,0.40,2.03,1488.317629
4,19.53,NaN,2.57,43.75,1.40,2.11,5261.094919


In [38]:
# Veremos metricas importantes como los quartiles, media y mediana ademas de la cantidad de nulos

df_original.describe()

,Por,LogPerm,AI,Brittle,TOC,VR,Production
count,879.000000,911.000000,899.000000,893.000000,907.000000,908.000000,1000.000000
mean,14.902366,1.406235,2.989911,49.424468,1.001996,1.987720,2247.295809
std,3.032083,0.407736,0.571529,14.929853,0.503468,0.311833,1464.256312
min,5.400000,0.120000,1.210000,3.030000,-0.260000,0.930000,2.713535
25%,12.790000,1.140000,2.580000,39.390000,0.630000,1.800000,1191.369560
50%,14.980000,1.390000,3.010000,49.440000,0.990000,1.990000,1976.487820
75%,17.060000,1.680000,3.375000,58.890000,1.370000,2.172500,3023.594214
max,24.650000,2.580000,4.700000,93.470000,2.710000,2.900000,12568.644130


In [36]:
# Que pocentaje de valores son nan por cada feature
features = df_original.columns
headers = ["Feature", "Missing Values", "% of Total Values"]
df_nan_features = df_original[features].isna().sum() / len(df_original) * 100
df_nan_features = pd.DataFrame({"Missing Values": df_original[features].isna().sum(), "% of Total Values": df_nan_features})
df_nan_features

,Missing Values,% of Total Values
Por,121,12.1
LogPerm,89,8.9
AI,101,10.1
Brittle,107,10.7
TOC,93,9.3
VR,92,9.2
Production,0,0.0


***
# END